In [16]:
import numpy as np
import datetime
import math
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
cust_df=pd.read_csv('Downloads/drive-download-20240215T093229Z-001/Customer_info.csv')
dscnt_df=pd.read_csv('Downloads/drive-download-20240215T093229Z-001/Discount_info.csv')
mkt_df=pd.read_csv('Downloads/drive-download-20240215T093229Z-001/Marketing_info.csv')
sales_df=pd.read_csv('Downloads/drive-download-20240215T093229Z-001/Onlinesales_info.csv')
tax_df=pd.read_csv('Downloads/drive-download-20240215T093229Z-001/Tax_info.csv')

In [18]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52924 entries, 0 to 52923
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   고객ID    52924 non-null  object 
 1   거래ID    52924 non-null  object 
 2   거래날짜    52924 non-null  object 
 3   제품ID    52924 non-null  object 
 4   제품카테고리  52924 non-null  object 
 5   수량      52924 non-null  int64  
 6   평균금액    52924 non-null  float64
 7   배송료     52924 non-null  float64
 8   쿠폰상태    52924 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 3.6+ MB


In [28]:
##제품카테고리 별로 고객군을 분류하는 함수
def rfm_function(product):
    sales_df=pd.read_csv('Downloads/drive-download-20240215T093229Z-001/Onlinesales_info.csv')
        
        # 데이터 필터링
    sales_df=sales_df[sales_df['수량']>0]
    sales_df=sales_df[sales_df['평균금액']>0]
    sales_df=sales_df[sales_df['배송료']>0]

        #거래날짜 데이터형식 datetime으로 변경
    sales_df['거래날짜 datetime']=sales_df['거래날짜'].apply(lambda x: pd.to_datetime(str(x),format='%Y-%m-%d'))
    
        #제품카테고리 선정 
    sales_df=sales_df[sales_df['제품카테고리']==product]
    
        #1인당 총구매금액 칼럼생성
    sales_df['총구매금액']=sales_df['수량']*sales_df['평균금액']
    
        #RFM 데이터 가공
    aggregations = {
        '거래날짜 datetime' : 'max',
        '거래ID' : 'count',
        '총구매금액' : 'sum'
    }
    rfm_df = sales_df.groupby('고객ID').agg(aggregations)

    rfm_df = rfm_df.rename(columns = {'거래날짜 datetime':'Recency',
                                     '거래ID':'Frequency',
                                     '총구매금액':'Monetary'}
                          )
    rfm_df=rfm_df.reset_index()
    
        #Recency 칼럼 값 가공 :오늘날짜(데이터 상 가장 최근 주문날짜)-고객 최근 주문날짜
    rfm_df['Recency']=datetime.datetime(2019,12,31) - rfm_df['Recency']
    rfm_df['Recency']=rfm_df['Recency'].apply(lambda x : x.days+1)

        #RFM 기준 나누기 (3분위수 활용)
    r_df=pd.qcut(rfm_df['Recency'],q=3,labels=['past','moderate','recent'])
    rfm_df['Recency기준']=r_df

    f_df=pd.qcut(rfm_df['Frequency'],q=3,labels=['low','moderate','high'])
    rfm_df['Frequency기준']=f_df

    q_df=pd.qcut(rfm_df['Monetary'],q=3,labels=['low','moderate','high'])
    rfm_df['Monetary기준']=q_df

    return rfm_df

In [32]:
##VIP고객, 이탈VIP고객, 잠재VIP고객, 이탈고객의 ID를 불러오는 함수

def VIP_function(rfm_df):
    ##VIP 고객 (Recency recent, Frequency high, Monetary high)
    VIP=rfm_df.loc[(rfm_df.Recency기준=='recent') & (rfm_df.Frequency기준=='high') & (rfm_df.Monetary기준=='high'),\
                      ['고객ID','Recency기준','Frequency기준','Monetary기준']]
    return VIP
 

def churned_VIP_function(rfm_df):
    ##이탈 VIP 고객 (Recency past, Frequency hihg, Monetary high)
    churned_VIP=rfm_df.loc[(rfm_df.Recency기준=='past') & (rfm_df.Frequency기준=='high') & (rfm_df.Monetary기준=='high'),\
                           ['고객ID','Recency기준','Frequency기준','Monetary기준']]
    return churned_VIP
    
def promising_function(rfm_df):
    ##잠재 VIP고객 (Recency recent, Frequency high or moderate, Monetary high or moderate)
    promising=rfm_df.loc[(rfm_df.Recency기준=='recent') & ~(rfm_df.Frequency기준=='low') & ~(rfm_df.Monetary기준=='low'),\
                         ['고객ID','Recency기준','Frequency기준','Monetary기준']]
    return promising

def churned_function(rfm_df):
    ##이탈 고객 (Recency past, Frequency low, Monetary low)
    churned=rfm_df.loc[(rfm_df.Recency기준=='past') & (rfm_df.Frequency기준=='low') & (rfm_df.Monetary기준=='low'),\
               ['고객ID','Recency기준','Frequency기준','Monetary기준']]
    return churned

In [40]:
#1인당 총구매금액 칼럼생성
sales_df['총구매금액']=sales_df['수량']*sales_df['평균금액']

#총구매금액을 기준으로 제품 나열
sales_df.groupby('제품카테고리')['총구매금액'].sum().sort_values(ascending=False)

제품카테고리
Nest-USA                2554202.39
Apparel                  591145.80
Nest                     518193.50
Office                   276794.40
Drinkware                200707.83
Bags                     151314.43
Notebooks & Journals     107085.96
Lifestyle                 74385.70
Nest-Canada               70910.40
Headgear                  53471.44
Gift Cards                19533.82
Google                     9420.47
Backpacks                  8772.69
Accessories                7295.88
Bottles                    6923.65
Waze                       6311.94
Fun                        6029.01
Housewares                 4637.32
More Bags                  2946.96
Android                     711.03
Name: 총구매금액, dtype: float64

In [112]:
#제품 Nest 고객 분류별 특성
product='Nest'
rfm_df=rfm_function(product)
rfm_df

,고객ID,Recency,Frequency,Monetary,Recency기준,Frequency기준,Monetary기준
0,USER_0001,60,2,628.00,moderate,low,moderate
1,USER_0004,108,2,398.00,recent,low,low
2,USER_0008,41,9,3744.00,moderate,high,high
3,USER_0012,123,3,597.00,recent,moderate,moderate
4,USER_0015,11,2,629.93,past,low,moderate
...,...,...,...,...,...,...,...
511,USER_1450,24,4,1269.04,past,moderate,high
512,USER_1453,114,2,995.00,recent,low,high
513,USER_1459,89,5,1343.00,recent,high,high
514,USER_1460,34,7,2189.00,moderate,high,high


In [113]:
VIP_function(rfm_df)

,고객ID,Recency기준,Frequency기준,Monetary기준
34,USER_0076,recent,high,high
36,USER_0078,recent,high,high
67,USER_0155,recent,high,high
113,USER_0300,recent,high,high
124,USER_0329,recent,high,high
132,USER_0343,recent,high,high
135,USER_0358,recent,high,high
158,USER_0418,recent,high,high
161,USER_0425,recent,high,high
164,USER_0435,recent,high,high


In [94]:
promising_function(rfm_df)

,고객ID,Recency기준,Frequency기준,Monetary기준
3,USER_0012,recent,moderate,moderate
10,USER_0030,recent,moderate,moderate
34,USER_0076,recent,high,high
36,USER_0078,recent,high,high
40,USER_0086,recent,moderate,moderate
...,...,...,...,...
489,USER_1374,recent,moderate,moderate
499,USER_1419,recent,moderate,moderate
503,USER_1433,recent,moderate,moderate
513,USER_1459,recent,high,high


In [96]:
churned_VIP_function(rfm_df).count()

고객ID           52
Recency기준      52
Frequency기준    52
Monetary기준     52
dtype: int64

In [98]:
churned_function(rfm_df).count()

고객ID           42
Recency기준      42
Frequency기준    42
Monetary기준     42
dtype: int64

In [51]:
#제품 Nest-USA 고객 분류별 특성
product='Nest-USA'
rfm_df=rfm_function(product)
rfm_df

,고객ID,Recency,Frequency,Monetary,Recency기준,Frequency기준,Monetary기준
0,USER_0001,60,20,3917.00,past,high,high
1,USER_0002,74,2,475.00,past,low,low
2,USER_0003,18,6,850.12,past,moderate,moderate
3,USER_0004,108,4,655.00,moderate,low,low
4,USER_0005,131,1,149.00,moderate,low,low
...,...,...,...,...,...,...,...
1255,USER_1460,34,24,4940.00,past,high,high
1256,USER_1461,19,3,515.78,past,low,low
1257,USER_1464,143,6,1259.00,moderate,moderate,moderate
1258,USER_1466,70,1,298.00,past,low,low


In [53]:
#제품 Nest-USA : VIP 인구통계학적 특징
vip_nest_usa=VIP_function(rfm_df)
vip_nest_usa

,고객ID,Recency기준,Frequency기준,Monetary기준
25,USER_0026,recent,high,high
58,USER_0064,recent,high,high
155,USER_0183,recent,high,high
184,USER_0217,recent,high,high
191,USER_0227,recent,high,high
...,...,...,...,...
1198,USER_1394,recent,high,high
1199,USER_1395,recent,high,high
1201,USER_1398,recent,high,high
1213,USER_1412,recent,high,high


In [57]:
pd.merge(left=vip_nest_usa, right=cust_df,how="inner",on='고객ID').groupby('고객지역')['고객ID'].count()

고객지역
California       26
Chicago          23
New Jersey        7
New York         20
Washington DC     8
Name: 고객ID, dtype: int64

In [58]:
pd.merge(left=vip_nest_usa, right=cust_df,how="inner",on='고객ID').groupby('성별')['고객ID'].count()

성별
남    32
여    52
Name: 고객ID, dtype: int64

In [76]:
#제품 Nest-USA : 잠재 VIP 인구통계학적 특징
promising_nest_usa=promising_function(rfm_df)
promising_nest_usa

,고객ID,Recency기준,Frequency기준,Monetary기준
23,USER_0024,recent,moderate,moderate
25,USER_0026,recent,high,high
27,USER_0029,recent,moderate,moderate
58,USER_0064,recent,high,high
74,USER_0085,recent,moderate,moderate
...,...,...,...,...
1221,USER_1421,recent,moderate,moderate
1228,USER_1428,recent,moderate,moderate
1233,USER_1434,recent,moderate,moderate
1238,USER_1440,recent,moderate,moderate


In [77]:
pd.merge(left=promising_nest_usa, right=cust_df,how="inner",on='고객ID').groupby('고객지역')['고객ID'].count()

고객지역
California       75
Chicago          59
New Jersey       20
New York         50
Washington DC    12
Name: 고객ID, dtype: int64

In [79]:
pd.merge(left=promising_nest_usa, right=cust_df,how="inner",on='고객ID').groupby('성별')['고객ID'].count()

성별
남     69
여    147
Name: 고객ID, dtype: int64

In [81]:
#제품 Nest-USA : 이탈 VIP 인구통계학적 특징
churned_vip_nest_usa=churned_VIP_function(rfm_df)
churned_vip_nest_usa

,고객ID,Recency기준,Frequency기준,Monetary기준
0,USER_0001,past,high,high
7,USER_0008,past,high,high
24,USER_0025,past,high,high
31,USER_0033,past,high,high
32,USER_0034,past,high,high
...,...,...,...,...
1216,USER_1415,past,high,high
1223,USER_1423,past,high,high
1234,USER_1435,past,high,high
1254,USER_1459,past,high,high


In [82]:
pd.merge(left=churned_vip_nest_usa, right=cust_df,how="inner",on='고객ID').groupby('고객지역')['고객ID'].count()

고객지역
California       47
Chicago          60
New Jersey       19
New York         39
Washington DC    10
Name: 고객ID, dtype: int64

In [83]:
pd.merge(left=churned_vip_nest_usa, right=cust_df,how="inner",on='고객ID').groupby('성별')['고객ID'].count()

성별
남     68
여    107
Name: 고객ID, dtype: int64

In [85]:
#제품 Nest-USA : 이탈고객의 인구통계학적 특징
churned_nest_usa=churned_function(rfm_df)
churned_nest_usa

,고객ID,Recency기준,Frequency기준,Monetary기준
1,USER_0002,past,low,low
6,USER_0007,past,low,low
16,USER_0017,past,low,low
35,USER_0037,past,low,low
57,USER_0063,past,low,low
...,...,...,...,...
1190,USER_1385,past,low,low
1208,USER_1405,past,low,low
1226,USER_1426,past,low,low
1256,USER_1461,past,low,low


In [86]:
pd.merge(left=churned_nest_usa, right=cust_df,how="inner",on='고객ID').groupby('고객지역')['고객ID'].count()

고객지역
California       29
Chicago          23
New Jersey       11
New York         19
Washington DC     5
Name: 고객ID, dtype: int64

In [87]:
pd.merge(left=churned_nest_usa, right=cust_df,how="inner",on='고객ID').groupby('성별')['고객ID'].count()

성별
남    24
여    63
Name: 고객ID, dtype: int64

In [131]:
# 제품 Nest-USA의 시기별 할인율
dscnt_df[(dscnt_df.제품카테고리=='Nest-USA')].sort_values(by='할인율')

,월,제품카테고리,쿠폰코드,할인율
3,Jan,Nest-USA,ELEC10,10
54,Apr,Nest-USA,ELEC10,10
105,Jul,Nest-USA,ELEC10,10
156,Oct,Nest-USA,ELEC10,10
4,Feb,Nest-USA,ELEC20,20
55,May,Nest-USA,ELEC20,20
106,Aug,Nest-USA,ELEC20,20
157,Nov,Nest-USA,ELEC20,20
5,Mar,Nest-USA,ELEC30,30
56,Jun,Nest-USA,ELEC30,30


In [45]:
#제품별 과세율
tax_df.sort_values(by='GST') 

,제품카테고리,GST
19,Nest,0.05
14,Gift Cards,0.05
13,Bottles,0.05
6,Notebooks & Journals,0.05
7,Headgear,0.05
17,Android,0.10
12,Google,0.10
11,Backpacks,0.10
10,Nest-Canada,0.10
0,Nest-USA,0.10


In [102]:
#VIP Nest-USA 고객별 쿠폰 사용 양상
vip_nest_usa=VIP_function(rfm_df)
vip_nest_usa

,고객ID,Recency기준,Frequency기준,Monetary기준
25,USER_0026,recent,high,high
58,USER_0064,recent,high,high
155,USER_0183,recent,high,high
184,USER_0217,recent,high,high
191,USER_0227,recent,high,high
...,...,...,...,...
1198,USER_1394,recent,high,high
1199,USER_1395,recent,high,high
1201,USER_1398,recent,high,high
1213,USER_1412,recent,high,high


In [104]:
pd.merge(left=vip_nest_usa,right=sales_df,how='inner',on='고객ID').groupby('쿠폰상태')['고객ID'].count()

쿠폰상태
Clicked     2810
Not Used     849
Used        1861
Name: 고객ID, dtype: int64

In [106]:
#잠재VIP Nest-USA 고객별 쿠폰 사용 양상
promising_nest_usa=promising_function(rfm_df)
promising_nest_usa

,고객ID,Recency기준,Frequency기준,Monetary기준
23,USER_0024,recent,moderate,moderate
25,USER_0026,recent,high,high
27,USER_0029,recent,moderate,moderate
58,USER_0064,recent,high,high
74,USER_0085,recent,moderate,moderate
...,...,...,...,...
1221,USER_1421,recent,moderate,moderate
1228,USER_1428,recent,moderate,moderate
1233,USER_1434,recent,moderate,moderate
1238,USER_1440,recent,moderate,moderate


In [107]:
pd.merge(left=promising_nest_usa,right=sales_df,how='inner',on='고객ID').groupby('쿠폰상태')['고객ID'].count()

쿠폰상태
Clicked     4623
Not Used    1372
Used        2980
Name: 고객ID, dtype: int64

In [108]:
#이탈 VIP Nest-USA 고객별 쿠폰 사용 양상
churned_vip_nest_usa=churned_VIP_function(rfm_df)
churned_vip_nest_usa

,고객ID,Recency기준,Frequency기준,Monetary기준
0,USER_0001,past,high,high
7,USER_0008,past,high,high
24,USER_0025,past,high,high
31,USER_0033,past,high,high
32,USER_0034,past,high,high
...,...,...,...,...
1216,USER_1415,past,high,high
1223,USER_1423,past,high,high
1234,USER_1435,past,high,high
1254,USER_1459,past,high,high


In [109]:
pd.merge(left=churned_vip_nest_usa,right=sales_df,how='inner',on='고객ID').groupby('쿠폰상태')['고객ID'].count()

쿠폰상태
Clicked     8505
Not Used    2561
Used        5610
Name: 고객ID, dtype: int64

In [110]:
#이탈 Nest-USA 고객별 쿠폰 사용 양상
churned_nest_usa=churned_function(rfm_df)
churned_nest_usa

,고객ID,Recency기준,Frequency기준,Monetary기준
1,USER_0002,past,low,low
6,USER_0007,past,low,low
16,USER_0017,past,low,low
35,USER_0037,past,low,low
57,USER_0063,past,low,low
...,...,...,...,...
1190,USER_1385,past,low,low
1208,USER_1405,past,low,low
1226,USER_1426,past,low,low
1256,USER_1461,past,low,low


In [111]:
pd.merge(left=churned_nest_usa,right=sales_df,how='inner',on='고객ID').groupby('쿠폰상태')['고객ID'].count()

쿠폰상태
Clicked     460
Not Used    149
Used        326
Name: 고객ID, dtype: int64